In [154]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re


df=pd.read_csv("dati_milano/milano_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [155]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['prezzo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','prezzo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','prezzo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)




In [156]:
# Visualizza lista ricerche in base ad orari
df['TIMESTAMP'].unique()

array(['03-02-2024---20:14', '03-02-2024---20:31', '03-02-2024---20:42',
       '03-02-2024---20:53', '03-02-2024---21:30', '03-02-2024---21:42',
       '03-02-2024---22:10', '03-02-2024---22:41', '03-03-2024---13:08',
       '03-03-2024---13:23'], dtype=object)

In [157]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[1]
orario2 = df["TIMESTAMP"].unique()[3]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [158]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[3:4] +df_list[5:-2]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
merged_df.columns

Index(['nome_hotel', 'prezzo_x', 'stanza_x', 'città_x', 'data',
       'numero_recensioni_x', 'distanza_centro_x', 'offerte_x',
       'colazione_inclusa_x', 'senza_pagamento_anticipato_x',
       'cancellazione_gratuita_x', 'os_x', 'user_agent_type_x', 'TIMESTAMP_x',
       'prezzo_mobile_x', 'offerta_super_segreta_x', 'offerta_inizio_2024_x',
       'prezzo_y', 'stanza_y', 'città_y', 'numero_recensioni_y',
       'distanza_centro_y', 'offerte_y', 'colazione_inclusa_y',
       'senza_pagamento_anticipato_y', 'cancellazione_gratuita_y', 'os_y',
       'user_agent_type_y', 'TIMESTAMP_y', 'prezzo_mobile_y',
       'offerta_super_segreta_y', 'offerta_inizio_2024_y', 'punteggio',
       'genius', 'username', 'booking_paga', 'offerta_tempo_limitato'],
      dtype='object')

In [ ]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y","offerte_x","offerte_y"]]

In [ ]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = merged_df["prezzo_x"]-merged_df["prezzo_y"]
merged_df.sort_values(by='differenza_prezzo',ascending= False)
merged_df[['nome_hotel','prezzo_x','prezzo_y','differenza_prezzo','prezzo_mobile_y','offerte_y']]



In [160]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
df_mobile = merged_df[(merged_df['prezzo_mobile_x'])|(merged_df['prezzo_mobile_y'])]
len(df_mobile)
len(merged_df)


353

In [163]:
# Elimino righe in cui il mobile non ha la stanza
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)

211